In [1]:
import pandas as pd
import numpy as np
import json
import re
train = pd.read_csv("train_v2.csv")
test = pd.read_csv("sampleSubmission_v2.csv")

In [2]:
chunk = [[],[],[],[],[],[]]
chunk[0] = open("../data/chunk0.json").read().split("\n")
chunk[1] = open("../data/chunk1.json").read().split("\n")
chunk[2] = open("../data/chunk2.json").read().split("\n")
chunk[3] = open("../data/chunk3.json").read().split("\n")
chunk[4] = open("../data/chunk4.json").read().split("\n")
chunk[5] = open("../data/chunk5.json").read().split("\n")

In [3]:
data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
data_special = []
cnt=0
for i in range(0,5):
    for st in chunk[i]:
        if st == "":continue
        if cnt%10000 == 0 : print cnt
        data = json.loads(st)
        id_s = ""
        for temp in data['id']:
            id_s+=temp
        data_id.append(id_s)
        data_images.append(len(data['images']))
        data_links.append(len(data['links']))
        title_s = ""
        for temp in data['title']:
            title_s+=temp
        data_title.append(title_s)
        data_text.append(len(data['text']))
        sentence = ""
        for temp in data['text']:
            sentence+="" + temp
        data_special.append(len(re.findall("[^a-zA-Z0-9_ ]",sentence)))
        data_text_sentence.append(sentence.strip())
        cnt+=1
        
train_all = pd.DataFrame({"id":data_id,
                          "images":data_images,
                          "links":data_links,
                          "title":data_title,
                          "text":data_text,
                          "sentence_text":data_text_sentence,
                          "special_len":data_special
                         })
train_all['file']=train_all['id'].apply(lambda x: x + '_raw_html.txt')
train_labeled = pd.merge(train,train_all,on='file')
train_labeled

data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
data_special = []
cnt=0
i=5
for st in chunk[i]:
    if st == "":continue
    if cnt%10000 == 0 : print cnt
    data = json.loads(st)
    id_s = ""
    for temp in data['id']:
        id_s+=temp
    data_id.append(id_s)
    data_images.append(len(data['images']))
    data_links.append(len(data['links']))
    title_s = ""
    for temp in data['title']:
        title_s+=temp
    data_title.append(title_s)
    data_text.append(len(data['text']))
    sentence = ""
    for temp in data['text']:
        sentence+="" + temp
    data_special.append(len(re.findall("[^a-zA-Z0-9_ ]",sentence)))
    data_text_sentence.append(sentence.strip())
    cnt+=1

sample_all = pd.DataFrame({"id":data_id,
                          "images":data_images,
                          "links":data_links,
                          "title":data_title,
                          "text":data_text,
                          "sentence_text":data_text_sentence,
                           "special_len":data_special
                         })
sample_all['file']=sample_all['id'].apply(lambda x: x + '_raw_html.txt')
test_labeled = pd.merge(test,sample_all,on='file')

train=[]
train_all=[]
test=[]
sample_all=[]
chunk = []
data_id = []
data_images = []
data_links = []
data_title = []
data_text = []
data_text_sentence = []
cnt=0

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
0
10000
20000
30000
40000
50000
60000


In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
def text_to_words(review):
    stemmer = PorterStemmer()
    letters_only = re.sub("[^a-zA-Z]"," ",review)
    
    words = letters_only.split()
    
    stops = set(stopwords.words("english"))
    
    meaningful_words = [w for w in words if not w in stops]
    
    singles = [stemmer.stem(w) for w in meaningful_words]
    
    return len(singles),(" ".join(singles))
    #return len(meaningful_words),(" ".join(meaningful_words))

clean_train_text = []
clean_len=[]

num_train = train_labeled["sentence_text"].size

for i in xrange(0,num_train):
    if i % 10000 == 0 : print i
    length, clean = text_to_words(train_labeled["sentence_text"][i])
    #clean = text_to_words(train_labeled["sentence_text"][i])
    clean_len.append(length)
    clean_train_text.append(clean)

train_labeled["sentence_text"]=clean_train_text
train_labeled["text_len"]=clean_len
clean_train_text=[]
    
clean_test_text = []
clean_len=[]

num_test = test_labeled["sentence_text"].size

for i in xrange(0,num_test):
    if i % 10000 == 0 : print i
    length, clean = text_to_words(test_labeled["sentence_text"][i])
    #clean = text_to_words(test_labeled["sentence_text"][i])
    clean_len.append(length)
    clean_test_text.append(clean)

test_labeled["sentence_text"]=clean_test_text
test_labeled["text_len"]=clean_len
clean_test_text=[]
clean_len=[]

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
0
10000
20000
30000
40000
50000
60000


In [18]:
for i in list(train_labeled):
    if(train_labeled[i].dtype == "int64"): train_labeled[i] = train_labeled[i].astype("int32")
for i in list(test_labeled):
    if(test_labeled[i].dtype == "int64"): test_labeled[i] = test_labeled[i].astype("int32")

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                            stop_words = None, max_features=5000)

train_data_features = vectorizer.fit_transform(train_labeled["sentence_text"])
train_data_features = train_data_features.toarray()
test_data_features = vectorizer.transform(test_labeled["sentence_text"])
test_data_features = test_data_features.toarray()

In [25]:
text_len=[]
for i in xrange(0,5000):
    train_labeled[str(i)]=train_data_features.T[i]
    train_labeled[str(i)]=train_labeled[str(i)].astype("int32")
    test_labeled[str(i)]=test_data_features.T[i]
    test_labeled[str(i)]=test_labeled[str(i)].astype("int32")

In [20]:
text_len = []
for i in xrange(0,num_train):
    text_len.append(train_labeled["text_len"][i]-sum(train_data_features[i]))
train_labeled["text_len"]=text_len
text_len = []
for i in xrange(0,num_test):
    text_len.append(test_labeled["text_len"][i]-sum(test_data_features[i]))
test_labeled["text_len"]=text_len

In [41]:
pre = list(train_labeled)
pre.remove("sponsored")
pre.remove("file")
pre.remove("id")
pre.remove("sentence_text")
pre.remove("title")
pre = ['images','links','special_len','text','text_len']
train_labeled[pre]

,images,links,special_len,text,text_len
0,4,57,44,6,28
1,1,0,0,0,0
2,0,0,0,0,0
3,26,87,8,3,5
4,23,71,176,42,143
5,37,96,341,58,242
6,45,91,5,1,0
7,7,77,9,6,4
8,8,100,22,11,20
9,9,17,42,4,7


In [42]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100,n_jobs=4)
#train_labeled[pre]
forest.fit(train_labeled[pre],train_labeled["sponsored"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
#test_data_features = vectorizer.transform(test_labeled["sentence_text"])
#test_data_features = test_data_features.toarray()
#result = forest.predict(test_data_features)
result = forest.predict_proba(test_labeled[pre])
print len(result[result==1])
print len(result[result==0])
result
#result = clf.predict(test_data_features)

11100
11100


array([[ 1.  ,  0.  ],
       [ 0.98,  0.02],
       [ 0.8 ,  0.2 ],
       ..., 
       [ 0.7 ,  0.3 ],
       [ 0.88,  0.12],
       [ 1.  ,  0.  ]])

In [47]:
output = pd.DataFrame({"file" : test_labeled["file"], "sponsored" : result.T[1]})
output

,file,sponsored
0,119_raw_html.txt,0.000000
1,167_raw_html.txt,0.020000
2,329_raw_html.txt,0.200000
3,419_raw_html.txt,0.060000
4,431_raw_html.txt,0.130000
5,455_raw_html.txt,0.050000
6,497_raw_html.txt,0.000000
7,527_raw_html.txt,0.150000
8,533_raw_html.txt,0.050000
9,575_raw_html.txt,0.040000


In [38]:
output.to_csv("bag_result6.csv",index=False,quoting=3)

In [38]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
kf = cross_validation.KFold(train_labeled.shape[0], n_folds=3)

for train_idx, test_idx in kf:
    X_train, X_test = (train_labeled[pre].values)[train_idx],(train_labeled[pre].values)[test_idx]
    Y_train, Y_test = train_labeled["sponsored"][train_idx],train_labeled["sponsored"][test_idx]
    #clf = BernoulliNB()
    #clf.fit(X_train,Y_train)
    #result=clf.predict(X_test)
    forest_imsi = RandomForestClassifier(n_estimators = 10)
    forest_imsi = forest_imsi.fit(X_train,Y_train)
    result=forest_imsi.predict(X_test)
    
    zero = (float)(len(result[(result==0)]))#/(float)(len(Y_test[Y_test==0]))
    one = (float)(len(result[(result==1)]))#/(float)(len(Y_test[Y_test==1]))
    
    print (zero,one)
    print (float)(len(Y_test[Y_test==0]))
    print (float)(len(Y_test[Y_test==1]))
    print len(Y_test)

(109598.0, 2744.0)
101458.0
10884.0
112342
(109507.0, 2834.0)
101435.0
10906.0
112341
(109565.0, 2776.0)
101452.0
10889.0
112341
